In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

In [4]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])


In [5]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./dataset/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./dataset/', train=False, transform=transforms.ToTensor())

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./dataset/cifar-10-python.tar.gz to ./dataset/


In [7]:
# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

In [8]:
# 3 x 3 convolution
def conv3x3(in_channels, out_channels, stride=1):
  return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                   stride=stride, padding=1, bias=False)


In [10]:
# Residual Block
class ResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1, downsample=None):
    super(ResidualBlock, self).__init__()
    self.conv1 = conv3x3(in_channels, out_channels, stride)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU(inplace=True)
    
    self.conv2 = conv3x3(out_channels, out_channels)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.downsample = downsample
    
  def forward(self, x):
    residual = x
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    
    out = self.conv2(out)
    out = self.bn2(out)
    
    if self.downsample:
      residual = self.downsample(x)
    
    out += residual
    out = self.relu(out)
    
    return out
    

In [11]:
# ResNet
class ResNet(nn.Module):
  def __init__(self, block, layers, num_classes=10):
    super(ResNet, self).__init__()
    self.in_channels = 16
    self.conv = conv3x3(in_channels=3, out_channels=16) 
    self.bn = nn.BatchNorm2d(num_features=16)
    self.relu = nn.ReLU(inplace=True)
    self.layer1 = self.make_layer(block, out_channels=16, blocks=layers[0], stride=1)
    self.layer2 = self.make_layer(block, out_channels=32, blocks=layers[1], stride=2)
    self.layer3 = self.make_layer(block, out_channels=64, blocks=layers[2], stride=2)
    self.avg_pool = nn.AvgPool2d(kernel_size=8)
    self.fc = nn.Linear(in_features=64, out_features=num_classes)
    
  def make_layer(self, block, out_channels, blocks, stride=1):
    downsample = None
    if (stride != 1) or (self.in_channels != out_channels):
      downsample = nn.Sequential(
        conv3x3(self.in_channels, out_channels, stride=stride),
        nn.BatchNorm2d(out_channels)
      )
    layers = []
    layers.append(block(self.in_channels, out_channels, stride, downsample))
    self.in_channels = out_channels
    for i in range(1, blocks):
      layers.append(block(out_channels, out_channels))
      
    return nn.Sequential(*layers)
  
  def forward(self, x):
    out = self.conv(x)
    out = self.bn(out)
    out = self.relu(out)
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.avg_pool(out)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out
      
    
    

In [12]:
model  = ResNet(block=ResidualBlock, layers=[2, 2, 2], num_classes=10).to(device)

In [14]:
# DataParallel 모듈로 모델 감싸기
model = nn.DataParallel(model)

In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

In [16]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [20]:
# Train the model
total_step = len(train_loader) # 500
curr_lr = learning_rate # 0.001

for epoch in range(num_epochs): #80
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    
  # Decay learning rate
  if (epoch+1) % 20 == 0:
    curr_lr /= 3
    update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.6644
Epoch [1/80], Step [200/500] Loss: 1.3710
Epoch [1/80], Step [300/500] Loss: 1.4131
Epoch [1/80], Step [400/500] Loss: 1.1597
Epoch [1/80], Step [500/500] Loss: 1.2825
Epoch [2/80], Step [100/500] Loss: 1.1434
Epoch [2/80], Step [200/500] Loss: 1.1014
Epoch [2/80], Step [300/500] Loss: 0.9884
Epoch [2/80], Step [400/500] Loss: 0.9012
Epoch [2/80], Step [500/500] Loss: 0.8901
Epoch [3/80], Step [100/500] Loss: 0.8069
Epoch [3/80], Step [200/500] Loss: 0.8655
Epoch [3/80], Step [300/500] Loss: 0.7975
Epoch [3/80], Step [400/500] Loss: 0.8003
Epoch [3/80], Step [500/500] Loss: 0.8313
Epoch [4/80], Step [100/500] Loss: 0.8385
Epoch [4/80], Step [200/500] Loss: 0.6672
Epoch [4/80], Step [300/500] Loss: 0.8027
Epoch [4/80], Step [400/500] Loss: 0.7519
Epoch [4/80], Step [500/500] Loss: 0.8633
Epoch [5/80], Step [100/500] Loss: 0.7148
Epoch [5/80], Step [200/500] Loss: 0.6327
Epoch [5/80], Step [300/500] Loss: 0.6583
Epoch [5/80], Step [400/500] Loss:

In [26]:
torch.set_printoptions(sci_mode=False)

# Test the model
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, dim=1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
  print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100])
outputs: torch.Size([100, 10])
predicted torch.Size([100

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')